#### model training:

In [1]:
import pandas as pd

data = pd.read_csv('data/^NDX_raw_data.csv')
data.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

data_backup = data.iloc[3524:]
data = data.iloc[:3524]

from statsmodels.tsa.seasonal import seasonal_decompose

period = 14
nb_last_bars = 10

decomposition_high = seasonal_decompose(data['high'], model='additive', period=period)
trend_high = decomposition_high.trend

decomposition_low = seasonal_decompose(data['low'], model='additive', period=period)
trend_low = decomposition_low.trend

data['trend_high'] = trend_high
data['trend_low'] = trend_low

import pandas as pd

for i in range(1,nb_last_bars+1):
    data[f'trend_h+{i}'] = data['trend_high'].shift(i)
for i in range(1,nb_last_bars+1):
    data[f'trend_l+{i}'] = data['trend_low'].shift(i)
data.dropna(axis=0, inplace=True)

#inputs_high = data.iloc[:, [8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]]
inputs_high = data.iloc[:, [8,9,10,11,12,13,14,15,16,17]]
target_high = data.iloc[:, [6]]

#inputs_low = data.iloc[:, [28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47]]
inputs_low = data.iloc[:, [18,19,20,21,22,23,24,25,26,27]]
target_low = data.iloc[:, [7]]

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor

X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(inputs_high, target_high, test_size=0.2, random_state=1, shuffle=False)
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(inputs_low, target_low, test_size=0.2, random_state=1, shuffle=False)

from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler_h = MinMaxScaler()
scaler_h.fit(X_train_h)
scaler_target_h = MinMaxScaler()
scaler_target_h.fit(y_train_h)

scaler_l = MinMaxScaler()
scaler_l.fit(X_train_l)
scaler_target_l = MinMaxScaler()
scaler_target_l.fit(y_train_l)

X_train_h_scaled = scaler_h.transform(X_train_h)
X_test_h_scaled = scaler_h.transform(X_test_h)
y_train_h_scaled = scaler_target_h.transform(y_train_h)
y_test_h_scaled = scaler_target_h.transform(y_test_h)

X_train_l_scaled = scaler_l.transform(X_train_l)
X_test_l_scaled = scaler_l.transform(X_test_l)
y_train_l_scaled = scaler_target_l.transform(y_train_l)
y_test_l_scaled = scaler_target_l.transform(y_test_l)

random_tree_high_20 = RandomForestRegressor(n_estimators=100, random_state=1)
random_tree_high_20.fit(X_train_h_scaled, y_train_h_scaled)

random_tree_low_20 = RandomForestRegressor(n_estimators=100, random_state=1)
random_tree_low_20.fit(X_train_l_scaled, y_train_l_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=1)

#### data reading for all stocks:

In [3]:
import os
import re

directory = 'data'

df_data = dict()

for file in os.listdir(directory):
    symbol_pattern = re.match(r'([^_]+)_', file)
    symbol = symbol_pattern.group(1)
    df_data[symbol] = pd.read_csv(os.path.join(directory, file))
    df_data[symbol].rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

In [4]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

period = 14
nb_last_bars = 10

for symbol in df_data:
    print(f'processing for {symbol}')
    
    # create columns with value 0
    for i in range(0,nb_last_bars+1):
        df_data[symbol][f'trend_h+{i}'] = 0.
    for i in range(0,nb_last_bars+1):
        df_data[symbol][f'trend_l+{i}'] = 0.
    
    for i in range(period*2, df_data[symbol].shape[0]):
        # decompose in trend_high
        decomposition_high_backup = seasonal_decompose(df_data[symbol]['high'].iloc[i-(period*2):i+1], model='additive', period=period)
        trend_high_backup = decomposition_high_backup.trend
    
        trend_high_backup.dropna(axis=0, inplace=True)
    
        for j in range(0,nb_last_bars+1):
            df_data[symbol].loc[i, f'trend_h+{j}'] = trend_high_backup.iloc[-1-j]
    
        # decompose in trend low
        decomposition_low_backup = seasonal_decompose(df_data[symbol]['low'].iloc[i-(period*2):i+1], model='additive', period=period)
        trend_low_backup = decomposition_low_backup.trend
    
        trend_low_backup.dropna(axis=0, inplace=True)
    
        for j in range(0,nb_last_bars+1):
            df_data[symbol].loc[i, f'trend_l+{j}'] = trend_low_backup.iloc[-1-j]

processing for AAPL
processing for AXP
processing for AMZN
processing for AMAT
processing for ABNB
processing for AVGO
processing for ABT
processing for ADI
processing for AMD
processing for ADP
processing for AMGN
processing for ^NDX
processing for ABBV
processing for ACN
processing for ADBE


In [5]:
for symbol in df_data:
    # drop the first rows of NaN trends
    df_data[symbol] = df_data[symbol].iloc[period*2:]
    df_data[symbol].reset_index(inplace=True)

In [6]:
df_data['AAPL'].shape

(5005, 29)

In [7]:
df_pred = dict()

count = 0

for symbol in df_data:

    count += 1

    #inputs_high_backup = df_data[symbol].iloc[:, [8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]]
    inputs_high_backup = df_data[symbol].iloc[:, [8,9,10,11,12,13,14,15,16,17]]
    #inputs_low_backup = df_data[symbol].iloc[:, [29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]]
    inputs_low_backup = df_data[symbol].iloc[:, [19,20,21,22,23,24,25,26,27,28]]

    print(count, symbol)
    
    scaler_input = MinMaxScaler()
    data_backup_h_scaled = scaler_input.fit_transform(inputs_high_backup)
    scaler_target = MinMaxScaler()
    target_backup_h_scaled = scaler_target.fit_transform(df_data[symbol][['trend_h+0']])

    y_pred_h_backup = random_tree_high_20.predict(data_backup_h_scaled)
    y_pred_h_backup = y_pred_h_backup.reshape(-1,1)
    y_pred_h_backup = scaler_target.inverse_transform(y_pred_h_backup)

    scaler_input = MinMaxScaler()
    data_backup_l_scaled = scaler_input.fit_transform(inputs_low_backup)
    scaler_target = MinMaxScaler()
    target_backup_l_scaled = scaler_target.fit_transform(df_data[symbol][['trend_l+0']])

    y_pred_l_backup = random_tree_low_20.predict(data_backup_l_scaled)
    y_pred_l_backup = y_pred_l_backup.reshape(-1,1)
    y_pred_l_backup = scaler_target.inverse_transform(y_pred_l_backup)

    dict_pred = {'pred_h':y_pred_h_backup, 'pred_l':y_pred_l_backup}
    df_pred[symbol] = dict_pred

1 AAPL
2 AXP
3 AMZN
4 AMAT
5 ABNB
6 AVGO
7 ABT
8 ADI
9 AMD
10 ADP
11 AMGN
12 ^NDX
13 ABBV
14 ACN
15 ADBE


In [8]:
count = 0

for symbol in df_data:
    count += 1

    #df_data[symbol].reset_index(inplace=True)

    from sklearn.metrics import mean_squared_error, accuracy_score, r2_score, mean_absolute_error
    
    score_h = mean_absolute_error(df_data[symbol][['high']], df_pred[symbol]['pred_h'])
    r2_h = r2_score(df_data[symbol][['high']], df_pred[symbol]['pred_h'])
    df_data[symbol]['mae_h'] = score_h
    
    score_l = mean_absolute_error(df_data[symbol][['low']], df_pred[symbol]['pred_l'])
    r2_l = r2_score(df_data[symbol][['low']], df_pred[symbol]['pred_l'])
    df_data[symbol]['mae_l'] = score_l
    
    print(f'{count} METRICS FOR {symbol}:')
    print(f'score high: {score_h}')
    print(f'score low: {score_l}')
    
    print(f'- R2 METRICS FOR {symbol}:')
    print(f'r2 high: {r2_h}')
    print(f'r2 low: {r2_l}')

1 METRICS FOR AAPL:
score high: 1.5518134724016024
score low: 1.5680704132702301
- R2 METRICS FOR AAPL:
r2 high: 0.9966014425456745
r2 low: 0.9965663762845842
2 METRICS FOR AXP:
score high: 2.464856674096579
score low: 2.4658695270333704
- R2 METRICS FOR AXP:
r2 high: 0.9900301044194336
r2 low: 0.9894373794392579
3 METRICS FOR AMZN:
score high: 1.802451977615845
score low: 1.8360055725924367
- R2 METRICS FOR AMZN:
r2 high: 0.9950156564326006
r2 low: 0.9948580329461061
4 METRICS FOR AMAT:
score high: 1.700329221406803
score low: 1.72658359333443
- R2 METRICS FOR AMAT:
r2 high: 0.9929232484841742
r2 low: 0.9925164346700281
5 METRICS FOR ABNB:
score high: 8.943012584551475
score low: 8.718621283026769
- R2 METRICS FOR ABNB:
r2 high: 0.8457012025838135
r2 low: 0.8364587621722419
6 METRICS FOR AVGO:
score high: 10.911998831382878
score low: 10.485620388755592
- R2 METRICS FOR AVGO:
r2 high: 0.9906567017003385
r2 low: 0.99143957099244
7 METRICS FOR ABT:
score high: 1.2290396795842193
score l

In [9]:
count = 0

remaining_columns = ('date','open','high','low','close','volume','trend_h+0','trend_l+0','mae_h','mae_l')

for symbol in df_data:
    count += 1

    df_data[symbol] = df_data[symbol][list(remaining_columns)]

    y_pred_h_series = pd.Series(df_pred[symbol]['pred_h'].reshape(1,-1)[0])
    y_pred_l_series = pd.Series(df_pred[symbol]['pred_l'].reshape(1,-1)[0])
    df_data[symbol]['pred_high'] = y_pred_h_series
    df_data[symbol]['pred_low'] = y_pred_l_series
    
    df_data[symbol].to_excel(f'data_w_pred_m5_3/{symbol}_w_pred_m5_3.xlsx')
    
    print(f'{count} done for {symbol}')

1 done for AAPL
2 done for AXP
3 done for AMZN
4 done for AMAT
5 done for ABNB
6 done for AVGO
7 done for ABT
8 done for ADI
9 done for AMD
10 done for ADP
11 done for AMGN
12 done for ^NDX
13 done for ABBV
14 done for ACN
15 done for ADBE


# model training for 1 stock:

In [14]:
import pandas as pd

data = pd.read_csv('data/^NDX_raw_data.csv')
data.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

data_backup = data.iloc[3524:]
data = data.iloc[:3524]

from statsmodels.tsa.seasonal import seasonal_decompose

period = 14
nb_last_bars = 10

decomposition_high = seasonal_decompose(data['high'], model='additive', period=period)
trend_high = decomposition_high.trend
detrend_high = decomposition_high.seasonal + decomposition_high.resid

decomposition_low = seasonal_decompose(data['low'], model='additive', period=period)
trend_low = decomposition_low.trend
detrend_low = decomposition_low.seasonal + decomposition_low.resid

data['trend_high'] = trend_high
data['trend_low'] = trend_low

#data['detrend_high'] = detrend_high
#data['detrend_low'] = detrend_low

import pandas as pd

for i in range(1,nb_last_bars+1):
    data[f'trend_h+{i}'] = data['trend_high'].shift(i)
for i in range(1,nb_last_bars+1):
    data[f'trend_l+{i}'] = data['trend_low'].shift(i)
data.dropna(axis=0, inplace=True)

inputs_high = data.iloc[:, [8,9,10,11,12,13,14,15,16,17]]
target_high = data.iloc[:, [6]]

inputs_low = data.iloc[:, [18,19,20,21,22,23,24,25,26,27]]
target_low = data.iloc[:, [7]]

In [15]:
data.columns, data.shape

(Index(['date', 'open', 'high', 'low', 'close', 'volume', 'trend_high',
        'trend_low', 'trend_h+1', 'trend_h+2', 'trend_h+3', 'trend_h+4',
        'trend_h+5', 'trend_h+6', 'trend_h+7', 'trend_h+8', 'trend_h+9',
        'trend_h+10', 'trend_l+1', 'trend_l+2', 'trend_l+3', 'trend_l+4',
        'trend_l+5', 'trend_l+6', 'trend_l+7', 'trend_l+8', 'trend_l+9',
        'trend_l+10'],
       dtype='object'),
 (3500, 28))

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor

X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(inputs_high, target_high, test_size=0.2, random_state=1, shuffle=False)
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(inputs_low, target_low, test_size=0.2, random_state=1, shuffle=False)

from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler_h = MinMaxScaler()
scaler_h.fit(X_train_h)
scaler_target_h = MinMaxScaler()
scaler_target_h.fit(y_train_h)

scaler_l = MinMaxScaler()
scaler_l.fit(X_train_l)
scaler_target_l = MinMaxScaler()
scaler_target_l.fit(y_train_l)

X_train_h_scaled = scaler_h.transform(X_train_h)
X_test_h_scaled = scaler_h.transform(X_test_h)
y_train_h_scaled = scaler_target_h.transform(y_train_h)
y_test_h_scaled = scaler_target_h.transform(y_test_h)

X_train_l_scaled = scaler_l.transform(X_train_l)
X_test_l_scaled = scaler_l.transform(X_test_l)
y_train_l_scaled = scaler_target_l.transform(y_train_l)
y_test_l_scaled = scaler_target_l.transform(y_test_l)

random_tree_high_10 = RandomForestRegressor(n_estimators=100, random_state=1)
random_tree_high_10.fit(X_train_h_scaled, y_train_h_scaled)

random_tree_low_10 = RandomForestRegressor(n_estimators=100, random_state=1)
random_tree_low_10.fit(X_train_l_scaled, y_train_l_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=1)

#### data reading for AAPL:

In [15]:
data_backup = pd.read_csv('data/AAPL_raw_data.csv')
data_backup.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

In [16]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

period = 14
nb_last_bars = 10

# create columns with value 0
for i in range(0,nb_last_bars+1):
    data_backup[f'trend_h+{i}'] = 0.
for i in range(0,nb_last_bars+1):
    data_backup[f'trend_l+{i}'] = 0.

for i in range(period*2, data_backup.shape[0]):
    # decompose in trend_high
    decomposition_high_backup = seasonal_decompose(data_backup['high'].iloc[i-(period*2):i+1], model='additive', period=period)
    trend_high_backup = decomposition_high_backup.trend
    detrend_high_backup = decomposition_high_backup.seasonal + decomposition_high_backup.resid

    trend_high_backup.dropna(axis=0, inplace=True)

    for j in range(0,nb_last_bars+1):
        data_backup.loc[i, f'trend_h+{j}'] = trend_high_backup.iloc[-1-j]

    # decompose in trend low
    decomposition_low_backup = seasonal_decompose(data_backup['low'].iloc[i-(period*2):i+1], model='additive', period=period)
    trend_low_backup = decomposition_low_backup.trend
    detrend_low_backup = decomposition_low_backup.seasonal + decomposition_low_backup.resid

    trend_low_backup.dropna(axis=0, inplace=True)

    for j in range(0,nb_last_bars+1):
        data_backup.loc[i, f'trend_l+{j}'] = trend_low_backup.iloc[-1-j]

# drop the first rows of NaN trends
data_backup = data_backup.iloc[period*2:]
data_backup.reset_index(inplace=True)

In [17]:
data_backup.columns, data_backup.shape

(Index(['index', 'date', 'open', 'high', 'low', 'close', 'volume', 'trend_h+0',
        'trend_h+1', 'trend_h+2', 'trend_h+3', 'trend_h+4', 'trend_h+5',
        'trend_h+6', 'trend_h+7', 'trend_h+8', 'trend_h+9', 'trend_h+10',
        'trend_l+0', 'trend_l+1', 'trend_l+2', 'trend_l+3', 'trend_l+4',
        'trend_l+5', 'trend_l+6', 'trend_l+7', 'trend_l+8', 'trend_l+9',
        'trend_l+10'],
       dtype='object'),
 (5005, 29))

In [18]:
df_pred = dict()

inputs_high_backup = data_backup.iloc[:, [8,9,10,11,12,13,14,15,16,17]]
inputs_low_backup = data_backup.iloc[:, [19,20,21,22,23,24,25,26,27,28]]

scaler_input = MinMaxScaler()
data_backup_h_scaled = scaler_input.fit_transform(inputs_high_backup)
scaler_target = MinMaxScaler()
target_backup_h_scaled = scaler_target.fit_transform(data_backup[['trend_h+0']])

y_pred_h_backup = random_tree_high_10.predict(data_backup_h_scaled)
y_pred_h_backup = y_pred_h_backup.reshape(-1,1)
y_pred_h_backup = scaler_target.inverse_transform(y_pred_h_backup)

scaler_input = MinMaxScaler()
data_backup_l_scaled = scaler_input.fit_transform(inputs_low_backup)
scaler_target = MinMaxScaler()
target_backup_l_scaled = scaler_target.fit_transform(data_backup[['trend_l+0']])

y_pred_l_backup = random_tree_low_10.predict(data_backup_l_scaled)
y_pred_l_backup = y_pred_l_backup.reshape(-1,1)
y_pred_l_backup = scaler_target.inverse_transform(y_pred_l_backup)

df_pred = {'pred_h':y_pred_h_backup, 'pred_l':y_pred_l_backup}

In [19]:
data_backup.reset_index(inplace=True)

from sklearn.metrics import mean_squared_error, accuracy_score, r2_score, mean_absolute_error

score_h = mean_absolute_error(data_backup[['high']], df_pred['pred_h'])
r2_h = r2_score(data_backup[['high']], df_pred['pred_h'])
data_backup['mae_h'] = score_h

score_l = mean_absolute_error(data_backup[['low']], df_pred['pred_l'])
r2_l = r2_score(data_backup[['low']], df_pred['pred_l'])
data_backup['mae_l'] = score_l

print(f'METRICS FOR NASDAQ INDEX:')
print(f'score high: {score_h}')
print(f'score low: {score_l}')

print(f'- R2 METRICS FOR NASDAQ INDEX:')
print(f'r2 high: {r2_h}')
print(f'r2 low: {r2_l}')

METRICS FOR NASDAQ INDEX:
score high: 1.5518134724016024
score low: 1.5680704132702301
- R2 METRICS FOR NASDAQ INDEX:
r2 high: 0.9966014425456745
r2 low: 0.9965663762845842


In [21]:
remaining_columns = ('date','open','high','low','close','volume','trend_h+0','trend_l+0','mae_h','mae_l')

data_backup = data_backup[list(remaining_columns)]

y_pred_h_series = pd.Series(df_pred['pred_h'].reshape(1,-1)[0])
y_pred_l_series = pd.Series(df_pred['pred_l'].reshape(1,-1)[0])
data_backup['pred_high'] = y_pred_h_series
data_backup['pred_low'] = y_pred_l_series

data_backup.to_excel(f'data_w_pred_m5_3/AAPL_w_pred_m5_3-10bar.xlsx')

/tmp/ipykernel_213683/1916406721.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_backup['pred_high'] = y_pred_h_series
/tmp/ipykernel_213683/1916406721.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_backup['pred_low'] = y_pred_l_series
